In [2]:
%matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pickle
import ge
import matplotlib.cm
from sklearn.neighbors import  KDTree

Using matplotlib backend: <object object at 0x00000292E1E8CF60>


In [36]:
"""
The two point clouds should be in the same coordinate system to be able to 
estimate the translations and rotations. It returns the translation and rotation
of the model point cloud.

The inputs are assumed to be shape Nx3.
"""
def ICP(
        source:np.array,
        model:np.array,
        min_err_change:float=0.5,
        max_iterations:int=20,
        sample_percentage:float=1.0
        )->np.array:
    prev_error=20000
    err_change=1000
    iter=0
    while(err_change>min_err_change):
        #sample random points from each of the pointsclouds, and create KDTree from the new samples
        sourceSamples=samplePoints(source,sample_percentage)
        treeModel=KDTree(model)
        #Get the indices of the nearest neighbours in the tree
        indices=treeModel.query(sourceSamples,k=1,return_distance=False)
        modelSamples=np.take(model,indices)
        modelSamples=np.reshape(modelSamples,(modelSamples.shape[0],modelSamples.shape[-1]))
        #Translate clouds by the means
        centroidM=np.mean(modelSamples,axis=0)
        centroidSource=np.mean(sourceSamples,axis=0)
        modelAligned=modelSamples-centroidM
        sourceAligned=sourceSamples-centroidSource
        #Use svd to get rotation and translation
        covarianceMatrix=modelAligned.T@sourceAligned
        print(covarianceMatrix.shape)
        U,S,Vt=np.linalg.svd(covarianceMatrix)
        rotation=U@Vt
        translation=centroidSource-rotation@centroidM
        source=rotation@source+translation

        #Calculate error, and the change in the error
        err=error(source,model)
        err_change=abs(err-prev_error)
        prev_error=err
        iter+=1
        if iter>max_iterations:
                break
    return rotation,translation

def samplePoints(array:np.array,sample_percentage):
    sampleIndices=np.random.choice(array.shape[0],int(sample_percentage*array.shape[0]),replace=False)
    samples=np.take(array,sampleIndices,axis=0)
    return samples

def error(source:np.array,model:np.array)->float:
    diffArray=source-model
    distanceArray=np.linalg.norm(diffArray,axis=1)
    mean=np.mean(distanceArray)
    return mean     

In [37]:
def toWorld(points,rotation,translation)->np.array:
    return rotation.T@(points-translation)

def processPCKLPointCloud(filePath, ICPTolerance=20):
    # load .pckl data from the RealSense sensor
    with open( filePath, "rb") as fh:
        data = pickle.load( fh )

    # a single view is stored in data[i]
    Nview = len( data ) # number of views

    # colormap for storing each view with different color
    cmap = matplotlib.cm.get_cmap( 'jet')

    # open PLY output object
    g = ge.GePly( filePath[:-5] + '.ply' ) # change extension .pckl to .ply

    referenceCloud=data[0]["points"].T
    rot=data[0]["extrinsics"][:3,:3]
    translation=data[0]["extrinsics"][:3,[3]]

    referenceCloud=toWorld(referenceCloud,rot,translation)
    C = cmap( 0 / ( Nview - 1 ) )[:3] # skip alpha
    g.points( referenceCloud.T, color=C )

    # iterate views
    for i in range(1,Nview):
        # 3D points from a single view - 3 x n numpy array
        X = data[i]["points"].T
        # rotation matrix, 3x3 numpy array
        R = data[i]['extrinsics'][:3, :3]
        # translation vector, 3x1 numpy array
        t = data[i]['extrinsics'][:3, [3]]

        # transform points to world coordinate system
        worldPoints=toWorld(X,R,t)
        # run ICP algorithm
        print(g.vertices[0].shape)
        rotatedPoints=ICP(g.vertices[0],worldPoints,ICPTolerance)

        # color for this view
        C = cmap( i / ( Nview - 1 ) )[:3] # skip alpha
        
        # ply output
        g.points( rotatedPoints, color=C )
    g.close()

In [38]:
filePath="Data/processing3DData/2019-12-09_11-06-02_data.pckl"
# load .pckl data from the RealSense sensor
with open( filePath, "rb") as fh:
    data = pickle.load( fh )

# a single view is stored in data[i]
Nview = len( data ) # number of views
cmap = matplotlib.cm.get_cmap( 'jet')
g = ge.GePly( filePath[:-5] + '.ply' ) # change extension .pckl to .ply

data0=data[0]["points"]
data1=data[1]["points"]
rotation,translation=ICP(data0,data1)


C:\Users\hgand\AppData\Local\Temp\ipykernel_10744\2063607977.py:8: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = matplotlib.cm.get_cmap( 'jet')


(1, 3)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 1)

In [6]:
print(data[0]["points"].shape)
print(data[1]["points"].shape)

(31127, 3)
(42229, 3)


In [9]:
U,S,Vt=np.linalg.svd(np.array([[1,1,1],[2,2,2],[3,3,3]]))
Vt.shape

(3, 3)